In [2]:
import sys
sys.path.append('../../../src')
import warnings
warnings.filterwarnings("ignore")


import os
import pandas as pd
import numpy as np
import torch
import pyreadr
import config
import Dataset
import time
import train_MLP
import argparse
import utils
import joblib

from Dataset import CNS
from torch.utils.data import DataLoader
from Model import DNAMLP
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from utils import make_ndarray_from_csv, get_int_label, brier_score_tensor
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score 
from torch.nn.functional import softmax, one_hot
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, RandomOverSampler

In [3]:
def train_epoch(epoch, model, train_loader, criterion, optimizer, device):
    correct = 0
    total = 0
    total_loss = 0
    total_bs = 0
    model.to(device)
    model.train()
    
    # For loop through all batches
    all_labels = []
    all_logits = []
    for features, labels in train_loader:
        # Move tensors to device
        features = features.to(device)
        labels = labels.to(device)
        
        # Zero out gradient
        optimizer.zero_grad()
        
        # Forward pass
        logits = model(features)
        # print(f'DJIWOEDJEWOD: {logits} - {labels}')
        # print(f'HDUHOAWDHEOE: {logits.shape} - {labels.shape}')
        loss = criterion(logits, labels)
        # print(logits.shape, labels.shape)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Evaluation
        total_loss += loss.item()
        _, predicted = logits.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
        # batch BS
        batch_bs = brier_score_tensor(logits, labels)
        total_bs += batch_bs
        
        # save logits and labels to calculate AUC
        for logit, label in zip(logits, labels):
            all_labels.append(label.item())
            all_logits.append(np.array(logit.detach().cpu().numpy()))
        
    # epoch's avrage LL
    train_loss = total_loss / len(train_loader)
    # epoch's average acc & ME
    train_acc = (correct / total) * 100.
    train_me = 100 - train_acc
    # epoch's average BS
    train_bs = total_bs/len(train_loader)
    # epoch's average AUC
    # all_labels_one_hot = one_hot(torch.Tensor(np.array(all_labels)).long())
    all_probs = softmax(torch.Tensor(np.array(all_logits)), dim = 1)
    # train_auc = roc_auc_score(all_labels_one_hot, all_probs)
    
    all_preds = [np.argmax(prob) for prob in all_probs]
    train_auc = roc_auc_score(all_labels, all_preds)
    train_precision = precision_score(all_labels, all_preds)
    train_recall = recall_score(all_labels, all_preds)
    
    return train_loss, train_acc, train_me, train_bs, train_auc, train_precision, train_recall

def val_epoch(epoch, model, val_loader, criterion, device):
    correct = 0
    total = 0
    total_loss = 0
    total_bs = 0
    model.to(device)
    # For loop through all batches
    with torch.no_grad():
        # For loop through all batches
        all_labels = []
        all_logits = []
        for features, labels in val_loader:
            # Move tensors to device
            features, labels = features.to(device), labels.to(device)
            
            # Forward pass
            logits = model(features)
            
            # Evaluation and batch loss
            loss = criterion(logits, labels)
            total_loss += loss.item()
            _, predicted = logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total  += labels.size(0)
            
            # batch BS
            batch_bs = brier_score_tensor(logits, labels)
            total_bs += batch_bs
            
            # save logits and labels to calculate AUC
            for logit, label in zip(logits,labels):
                all_labels.append(label.item())
                all_logits.append(np.array(logit.detach().cpu().numpy()))
        
        # epoch's average LL
        val_loss = total_loss / len(val_loader)
        # epoch's average acc & ME
        val_acc = (correct / total) * 100
        val_me = (100 - val_acc)
        # epoch's average BS
        val_bs = total_bs/len(val_loader)
        # epoch's AUC
        # all_labels_one_hot = one_hot(torch.Tensor(np.array(all_labels)).long())
        all_probs = softmax(torch.Tensor(np.array(all_logits)), dim = 1)
        # val_auc = roc_auc_score(all_labels_one_hot, all_probs)
        
        all_preds = [np.argmax(prob) for prob in all_probs]
        val_auc = roc_auc_score(all_labels, all_preds)
        val_precision = precision_score(all_labels, all_preds)
        val_recall = recall_score(all_labels, all_preds)
         
    return val_loss, val_acc, val_me, val_bs, val_auc, val_precision, val_recall

def run(class_name, fold, train_loader, val_loader, model, criterion, optimizer, config, save):
    history = {'train_accs': [], 'train_losses': [], 'val_accs': [], 'val_losses': []}
    
    model.to(config['device'])
    n_epochs = config['mlp_n_epochs']
    BEST_STATES_DIR= config['MLP_BEST_STATES_DIR']
    BEST_STATE_PATH = os.path.join(BEST_STATES_DIR, class_name, f'{fold}.pth')
    diff_threshold = config['mlp_diff_threshold']
    max_patience = config['mlp_max_patience']
    patience = 0
    
    for epoch in range(1, n_epochs + 1):
        
        train_loss, train_acc, train_me, train_bs, train_auc, train_precision, train_recall = train_epoch(epoch, model, train_loader, criterion, optimizer, config['device'])
        val_loss, val_acc, val_me, val_bs, val_auc, val_precision, val_recall = val_epoch(epoch, model, val_loader, criterion, config['device'])
        
        history['train_accs'].append(train_acc)
        history['train_losses'].append(train_loss)
        history['val_accs'].append(val_acc)
        history['val_losses'].append(val_loss)
        
        print(f'{class_name.upper()}] - {fold} - {epoch}/{n_epochs}')
        print('train_loss: %.5f | train_acc: %.3f | train_precision: %.3f | train_recall: %.3f | train_auc: %.3f' % (train_loss, train_acc, train_precision, train_bs, train_auc))
        print('val_loss: %.5f | val_acc: %.3f | val_precision: %.3f | val_recall: %.3f | val_auc: %.3f' % (val_loss, val_acc, val_precision, val_recall, val_auc))
        
        eval_file = open(config['MLP_EVALUATION_RESULTS'], 'a+')
        eval_file.write(
            f'\n>>>>>>[{class_name.upper()}] - {fold} - {epoch}/{n_epochs}: train_loss: %.5f | train_acc: %.3f | train_precision: %.3f | train_recall: %.3f | train_auc: %.3f\nval_loss: %.5f | val_acc: %.3f | val_precision: %.3f | val_recall: %.3f | val_auc: %.3f\n' % (train_loss, train_acc, train_precision, train_recall, train_auc, val_loss, val_acc, val_precision, val_recall, val_auc)
        )
        eval_file.close()
            
        if val_loss == min(history['val_losses']):
            if save.lower() == 'save':
                print('Lowest validation loss => saving model weights...')
                torch.save(model.state_dict(), BEST_STATE_PATH)
        if len(history['val_losses']) > 1:
            if abs(history['val_losses'][-2] - val_loss) < diff_threshold or history['val_losses'][-2] < val_loss:
                patience = patience + 1
                print(f'Patience increased to {patience}')
                if patience == max_patience:
                    print('Early stopping.')
                    break
            else:
                patience = 0
        print('---------------------------------------------')
    return max(history['val_accs'])

In [4]:
print("Running mlp classifiers")
clf_cfg = config.classifier_config
save = 'no_save'

print(f'root: {config.root_dir}')
print(f"device: {clf_cfg['device']}")
print(f'save mode: {save}')

folds = utils.inner_folds
groups = utils.positive_groups

fold = folds[np.random.randint(len(folds))]
group = groups[np.random.randint(len(groups))]
print(f'{group} - {fold}')

# Read from csv to dataframe
features, labels = make_ndarray_from_csv(group, fold, mode = 'all')
features.shape, labels.shape

Running mlp classifiers
root: /media/data/hungnt/work/SourceCode/Brain_DNA/src/..
device: cuda
save mode: no_save
Nerve - 5.1


((127, 10000), (127,))

In [5]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.5, random_state=42)
train_features.shape, val_features.shape

((63, 10000), (64, 10000))

In [6]:
smote = SMOTE(sampling_strategy = "auto", random_state = 42, k_neighbors = 4)
new_train_features, new_train_labels = smote.fit_resample(train_features, train_labels)
new_train_features.shape

(100, 10000)

In [7]:
# Encode the labels
new_train_labels_int = np.array(
    [get_int_label(label, group) for label in new_train_labels])
val_labels_int = np.array(
    [get_int_label(label, group) for label in val_labels])

# Create datasets and Dataloaders
train_dataset = CNS(new_train_features, new_train_labels_int, mode='train')
val_dataset = CNS(val_features, val_labels_int, mode='val')
train_loader = DataLoader(
    train_dataset, batch_size=clf_cfg['mlp_train_batch_size'], shuffle=True)
val_loader = DataLoader(
    val_dataset, batch_size=clf_cfg['mlp_val_batch_size'], shuffle=False)

# Init model object
in_features = clf_cfg['n_features']
model = DNAMLP(in_features, clf_cfg['n_classes'])
if clf_cfg['MLP_FIRST_TIME'] == False:
    # Load model based on fold
    BEST_STATE_PATH = os.path.join(
        clf_cfg['MLP_BEST_STATES_DIR'], group, f'{fold}.pth')
    model.load_state_dict(torch.load(BEST_STATE_PATH))

# Define training and validating hyperparams
criterion = CrossEntropyLoss(weight=None)
optimizer = Adam(model.parameters(
), lr=clf_cfg['mlp_lr'], weight_decay=clf_cfg['mlp_weight_decay'])
print(f'Running in {save} mode')
best_accs = run(
    group, fold, train_loader, val_loader, model, criterion, optimizer, clf_cfg, save)

Running in no_save mode
NERVE] - 5.1 - 1/30
train_loss: 0.56632 | train_acc: 76.000 | train_precision: 0.760 | train_recall: 0.379 | train_auc: 0.760
val_loss: 0.32810 | val_acc: 98.438 | val_precision: 0.981 | val_recall: 1.000 | val_auc: 0.962
---------------------------------------------
NERVE] - 5.1 - 2/30
train_loss: 0.35567 | train_acc: 91.000 | train_precision: 0.956 | train_recall: 0.202 | train_auc: 0.910
val_loss: 0.36141 | val_acc: 82.812 | val_precision: 1.000 | val_recall: 0.784 | val_auc: 0.892
Patience increased to 1
---------------------------------------------
NERVE] - 5.1 - 3/30
train_loss: 0.24475 | train_acc: 96.000 | train_precision: 1.000 | train_recall: 0.114 | train_auc: 0.960
val_loss: 0.19425 | val_acc: 100.000 | val_precision: 1.000 | val_recall: 1.000 | val_auc: 1.000
---------------------------------------------
NERVE] - 5.1 - 4/30
train_loss: 0.19256 | train_acc: 99.000 | train_precision: 1.000 | train_recall: 0.081 | train_auc: 0.990
val_loss: 0.20208 | v

In [8]:
print(best_accs)

100.0
